In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv

In [ ]:
def configure():
    load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('openai_api_key')

In [ ]:
def load_and_prepare_documents(file_path):
    loader = TextLoader(file_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    return docs

In [ ]:
def create_vector_store(docs):
    """Embed the documnets and store them in FAISS for retrieval."""
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(docs, embeddings)
    return vector_store

In [ ]:
def ask_questions(vector_store, query):
    """Retrieve relevant chunks and generate responses using LLM."""
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.7)

    system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)

    retriever = vector_store.as_retriever()
    
    chain = create_retrieval_chain(retriever, question_answer_chain)
    chain.invoke({"input": query})

In [ ]:
if __name__ == "__main__":
    file_path = 'sample.txt'
    
    print('loading and processing documents...')
    documents = load_and_prepare_documents(file_path)

    print('creating vector store...')
    vector_store = create_vector_store(documents)

    while True:
        query = input("Ask a question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break

        response = ask_questions(vector_store, query)
        print(response)